In [ ]:
FileList = ['20010114-Remarks at the National Day of Prayer & Remembrance Service.txt',
            '20010115-First Radio Address following 911.txt',
            '20010117-Address at Islamic Center of Washington, D.C..txt',
           '20010120-Address to Joint Session of Congress Following 911 Attacks.txt',
           '20010911-Address to the Nation.txt',
           '20011007-Operation Enduring Freedom in Afghanistan Address to the Nation.txt',
           '20011011-911 Pentagon Remembrance Address.txt',
           '20011011-Prime Time News Conference on War on Terror.txt',
           '20011026-Address on Signing the USA Patriot Act of 2001.txt',
           '20011110-First Address to the United Nations General Assembly.txt',
           '20011211-Address to Citadel Cadets.txt',
           '20011211-The World Will Always Remember 911.txt',
           '20020129-First (Official) Presidential State of the Union Address.txt',
           ]
doc = ''

for f in FileList:
    text = open('C:/Users/Steve/Documents/Cultural Violence/George Bush/' + f, 'r')
    doc = doc + text.read()
    text.close

In [39]:
# open file
print('Opening document')
text = open('C:/Users/Steve/Documents/Cultural Violence/GBSpeech.txt', 'r')
doc = text.read()
text.close()
print('complete')

Opening document
complete


In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup

page_link = 'https://www.americanrhetoric.com/speeches/gwbush911addresstothenation.htm'
page_response = requests.get(page_link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")

textContent = []
for i in range(20, 40):
    paragraphs = page_content.find_all("br")[i].text
    textContent.append(paragraphs)
    
print(textContent)

In [40]:
import spacy
from spacy.pipeline import merge_entities


print('setting up pipeline')
coref_nlp = spacy.load('en_coref_md')
spacy_nlp = spacy.load('en_core_web_md')

spacy_nlp.add_pipe(merge_entities)

print('applying pipelines')
coref_doc = coref_nlp(doc)
new_doc = spacy_nlp(coref_doc._.coref_resolved) #replace co-reference entities with root reference

print('complete')


setting up pipeline
applying pipelines
complete


In [41]:
print(coref_doc._.coref_resolved)

Mr. Speaker, Mr. President Pro Tempore, members of Congress, and fellow Americans:
In the normal course of events, Presidents come to this chamber to report on the state of the Union. Tonight, no such report is needed. no such report has already been delivered by the American people.
We have seen no such report in the courage of passengers, who rushed terrorists to save others on the ground -- passengers like an exceptional man named Todd Beamer. And would you please help me to welcome an exceptional man named Todd Beamer wife, Lisa Beamer, here Tonight. (Applause.)
We have seen the state of We Union in the endurance of rescuers, working past exhaustion. We have seen the unfurling of flags, the lighting of candles, the giving of blood, the saying of prayers -- in English, Hebrew, and Arabic. We have seen the decency of a loving and giving people who have made the grief of strangers the grief of strangers own.
My fellow citizens, for the last nine days, the entire world has seen for the

In [ ]:
#extract named entities

named_entities = [x.text for x in new_doc.ents]

In [38]:
from spacy import displacy
from pathlib import Path

# displacy.render(new_doc, jupyter=True, style='ent')

html = displacy.render(new_doc, jupyter=True, style='dep', page=True)
file_name = 'GB Speech' + '.html'
output_path = Path('C:/Users/Steve/Documents/Cultural Violence/George Bush/' + file_name)
output_path.open('w', encoding='utf-8').write(html)

#sentences = [x for x in new_doc.sents]

#for sentence in sentences[0:5]:
#    displacy.render(sentence, jupyter=True, style='ent')
    
# ent = entity resolution
# dep = dependency resolution

TypeError: write() argument must be str, not None

In [7]:
#clean document

entity_list = []
doc_array = []

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

print('Tokenizing document')

for word in spacy_nlp.Defaults.stop_words:
    lex = spacy_nlp.vocab[word]
    lex.is_stop = True

for sentence in new_doc.sents:
    doc_array.append([lemmatizer.lemmatize(token.text).strip().lower() for token in sentence if 
                      (not token.is_punct and token.text.find('\n') and not token.is_space and token.text !="'s")])
    
# not token.is_stop and
    
print('complete')
for i in doc_array:
    print(i)

Tokenizing document
complete
['mr.', 'speaker', 'mr.', 'president', 'pro tempore', 'member', 'of', 'congress', 'and', 'fellow', 'americans']
['in', 'the', 'normal', 'course', 'of', 'event', 'presidents', 'come', 'to', 'this', 'chamber', 'to', 'report', 'on', 'the', 'state', 'of', 'the', 'union']
['tonight', 'no', 'such', 'report', 'is', 'needed']
['no', 'such', 'report', 'ha', 'already', 'been', 'delivered', 'by', 'the', 'american', 'people']
['we', 'have', 'seen', 'no', 'such', 'report', 'in', 'the', 'courage', 'of', 'passenger', 'who', 'rushed', 'terrorist', 'to', 'save', 'others', 'on', 'the', 'ground', 'passenger', 'like', 'an', 'exceptional', 'man', 'named', 'todd beamer']
['and', 'would', 'you', 'please', 'help', 'me', 'to', 'welcome', 'an', 'exceptional', 'man', 'named', 'todd beamer', 'wife', 'lisa beamer', 'here', 'tonight']
['applause']
['we', 'have', 'seen', 'the', 'state', 'of', 'we union', 'in', 'the', 'endurance', 'of', 'rescuer', 'working', 'past', 'exhaustion']
['we', '

In [ ]:
print('Creating Word2Vec model')
from gensim.models import Word2Vec
model = Word2Vec(doc_array, size=500, window=5, min_count=1, workers=4, sg=0)
print('complete')

In [ ]:
print('Create a list of named entities') #does this need to be named entities or mentions?
    
# document level
entities = [(e.text, e.start_char, e.end_char, e.label_) for e in new_doc.ents]

# entity level
entity_list = []

for entity in entities:
    if entity[3] in ['PERSON', 'ORG', 'NORP', 'GPE'] and entity[0] != '\n':
        entity_list.append(entity[0].lower())
        
entity_set = set(entity_list)
print('complete')

In [ ]:
print('Creating a list of influence words')

# this is a set of elevation and otherising words. The idea is to measure the proximity of these words to the named entities

biblical_set = {'god', 'evil'}
ideology_set = {'nazism'}
crime_set = {'murder', 'murderers', 'murderous', 'terrorist', 'terrorism'}
total_set = biblical_set | entity_set | ideology_set | crime_set

print('complete')

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model, set_array):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    print('creating tokens and labels')
    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    print('building tsne model')
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)
    
    print('constructing graph')
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        
        if labels[i] in set_array:
            plt.annotate(labels[i],
                xy=(x[i], y[i]),
                xytext=(5, 2),
                textcoords='offset points',
                ha='right',
                va='bottom')
    plt.show()
    
tsne_plot(model, total_set)

In [ ]:
#model.wv.most_similar_cosmul(positive=['taliban', 'america'], negative = ['enemy'])

model.wv.most_similar('them')


In [ ]:
model.wv.similarity('enemy', 'al qaeda')

In [ ]:
from gensim.summarization import keywords

keyword_array = []

#words = keywords(doc[0])
#words = words.split('\n')

print(keywords(doc))